In [1]:
import pandas as pd
import numpy as np
from preprocess import document_preprocess


from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from scipy.spatial import distance
from progressbar import ProgressBar
pbar = ProgressBar()

In [2]:
data = pd.read_csv(r"data\Lens-AU.csv")
train = list((data['Abstract']+data['Title']).values.astype('U'))
train=train[:5000]

In [3]:
ob=document_preprocess(lemmatize=True, stop_words=True, singleton=True, custom_stop_words=[])

In [4]:
preprocessed_doc=[]

for text in pbar(train):
    filtered_list=ob.filter_word(text)
    filtered_string =' '.join([str(item) for item in filtered_list])
    preprocessed_doc.append(filtered_string)

100% |########################################################################|


In [5]:
vectorizer = TfidfVectorizer(ngram_range=(2,2))
X = vectorizer.fit_transform(preprocessed_doc).todense()

df=pd.DataFrame(X, columns=vectorizer.get_feature_names())

In [6]:
def cos_sim(vector1, vector2):
    cosine_similarity = 1 - distance.cosine(vector1, vector2)
    return cosine_similarity

In [7]:
dict_={}

for i in df.columns:
    dict_[i] = cos_sim(df[i], df["artificial intelligence"])

In [8]:
dict_ = dict(sorted(dict_.items(), key=lambda item: item[1], reverse=True))

results=pd.DataFrame(dict_, index=[0]).T
results.to_csv("AI_cosine_similarity_results_tfidf.csv")